# Import libraries

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

# Load data

Load data and take a look at what we have.

In [2]:
store_data = pd.read_csv("./data/store.csv")
sales_data = pd.read_csv("./data/train.csv")

/Users/corstiaen/opt/anaconda3/envs/minicompenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(f"Number of samples in entire dataset: {len(sales_data)}")
sales_data.head()

Number of samples in entire dataset: 637774


,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0


In [4]:
print(f"Length of store-table: {len(store_data)}")
store_data.head()

Length of store-table: 1115


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# Preprocessing of data

## First cleaning

Samples without sales data as well as days that sales are zero can be dropped.

In [5]:
sales_data.dropna(subset=["Sales"], inplace=True)

keep_mask = sales_data.loc[:, "Sales"] != 0
sales_data = sales_data[keep_mask]

We will also drop the customers feature. This feature is directly linked to our target, the sales. Moreover the number of customers in the future is not known, again similar to the sales themselves.

In [6]:
sales_data.drop("Customers", axis=1, inplace=True)

After these steps there are still 30 duplicated samples in our dataset. Although these duplicates may actually be real numbers af two shops having the exact same sales on a single day, it is hard to disentangle. Moreover, 30 samples on a dataset of >500,000 is negligible.

In [7]:
duped_mask = sales_data.duplicated()
duplicated_data = sales_data.loc[duped_mask, :]

print(len(duplicated_data))
duplicated_data.head()

30


,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday
17319,2013-01-16,NaN,3.0,2795.0,1.0,0.0,0,0.0
43383,2013-02-08,NaN,5.0,5706.0,1.0,1.0,0,0.0
79395,2013-03-13,NaN,3.0,4907.0,1.0,0.0,0,0.0
83406,2013-03-16,NaN,6.0,4648.0,1.0,0.0,0,0.0
122096,2013-04-20,NaN,6.0,2890.0,1.0,0.0,0,0.0


In [8]:
sales_data.drop_duplicates(inplace=True)

## Merging sales and store data

We need to merge the sales data with the store data. Checking data types reveals that the feature on which we will merge, "Store", is not of the same type. We change the datatype of "Store" in the sales data, such that it matches the store data. We do this step manually here, but this step will also be included in our pipeline.

In [9]:
sales_data.dtypes

Date              object
Store            float64
DayOfWeek        float64
Sales            float64
Open             float64
Promo            float64
StateHoliday      object
SchoolHoliday    float64
dtype: object

In [10]:
store_data.dtypes

Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

There's NaNs in the store ID. We will encode them with 0, because that label does not exist for a store in the store data.

In [11]:
sales_data.isnull().sum()

Date                 0
Store            15550
DayOfWeek        15299
Sales                0
Open             15455
Promo            15439
StateHoliday     15560
SchoolHoliday    15547
dtype: int64

In [12]:
store_data.loc[:, "Store"].min()

1

In [13]:
sales_data = sales_data.fillna({"Store": 0})

In [14]:
sales_data = sales_data.astype({"Store": int})

Actual merging of tables.

In [15]:
merged_data = sales_data.merge(store_data, how="left", on="Store")

In [16]:
print(f"The total number of samples in our dataset is {len(merged_data)}")
merged_data.head()

The total number of samples in our dataset is 512926


,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2013-01-01,353,2.0,3139.0,1.0,0.0,a,1.0,b,b,900.0,NaN,NaN,1.0,14.0,2013.0,"Feb,May,Aug,Nov"
1,2013-01-01,335,2.0,2401.0,1.0,0.0,a,1.0,b,a,90.0,NaN,NaN,1.0,31.0,2013.0,"Jan,Apr,Jul,Oct"
2,2013-01-01,512,2.0,2646.0,1.0,0.0,a,1.0,b,b,590.0,NaN,NaN,1.0,5.0,2013.0,"Mar,Jun,Sept,Dec"
3,2013-01-01,494,2.0,3113.0,1.0,0.0,a,1.0,b,a,1260.0,6.0,2011.0,0.0,NaN,NaN,NaN
4,2013-01-01,530,2.0,2907.0,1.0,0.0,a,1.0,a,c,18160.0,NaN,NaN,0.0,NaN,NaN,NaN


# Splitting train, validation, and test sets

When using tree-based models we can use random splitting of the data:

In [17]:
X = merged_data.loc[:, ~merged_data.columns.isin(["Sales"])]
y = merged_data.loc[:, "Sales"]

X_train_valid, X_test, y_train_valid, y_test = train_test_split(
    X, y,
    random_state=42,
    train_size=.8,
)

X_train, X_validation, y_train, y_validation = train_test_split(
    X_train_valid,
    y_train_valid,
    random_state=42,
    train_size=.8,
)

In [18]:
print(f"Our train set has {len(X_train)} samples")
print(f"Our validation set has {len(X_validation)} samples")
print(f"Our test set has {len(X_test)} samples")
X_train.head()

Our train set has 328272 samples
Our validation set has 82068 samples
Our test set has 102586 samples


,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
143140,2013-06-12,353,3.0,1.0,0.0,0,0.0,b,b,900.0,NaN,NaN,1.0,14.0,2013.0,"Feb,May,Aug,Nov"
494300,2014-07-08,267,2.0,1.0,0.0,0,0.0,c,a,2460.0,1.0,2012.0,0.0,NaN,NaN,NaN
417249,2014-04-08,734,2.0,1.0,0.0,0.0,0.0,a,a,220.0,NaN,NaN,1.0,36.0,2013.0,"Mar,Jun,Sept,Dec"
370025,2014-02-17,0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249373,2013-10-05,510,NaN,1.0,NaN,0,0.0,a,c,8260.0,NaN,NaN,0.0,NaN,NaN,NaN


**When using trend+seasonality based models we need to split the data using a date-cutoff. This may be implemented later.**

# Data profiling

The cell below is commented out for faster running of this notebook. The resulting profile report can be found by following this link:

**[Rossmann profile report](files/rossmann_profile_report.html)**

In [19]:
profile = ProfileReport(X_train)
profile.to_file(output_file="rossmann_profile_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/corstiaen/opt/anaconda3/envs/minicompenv/lib/python3.8/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Feature engineering

### Check amount of features missing per sample. If 4 or more, meaning about 50%, try dropping entire row.

### We also need to look into outliers!

**Lilla**

    Date -> datetime
    split in year, month, week, day of week; drop the original
    make sure weekday matches weekday from date; check for missing values and impute where necessary
    encode all except year with sine+cosine! https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/

    DayOfWeek
    drop, replace with values derived from Date

    CompetitionDistance -> float
    Try: median imputation
    Try also: KNNImputer (sklearn)

**Corstiaen**

    Store
    3 options:
    * drop
    * target encoding -> float
    * frequency encoding -> int
    test model to seae what works best
   **MAKE SURE STORE FEATURE IS SAME dTYPE;
   NEEDS 0 IMPUTING FOR THAT**

    Open -> one-hot encode
    3 categories, meaning 2 one-hot columns
    Started with median imputing (1 for True)
    
    Promo -> one-hot encode
    3 categories, meaning 2 one-hot columns
    Started with median imputing (0 for False)
    
    StateHoliday -> one-hot encode
    5 categories, meaning 4 one-hot columns
    turn 0.0 into 0
    ['0', 0.0, 'c', nan, 'a', 'b']
    Started with median imputing (0 for No Holiday)
    
    SchoolHoliday -> one-hot encode
    3 categories, meaning 2 one-hot columns
    Started with median imputing (0 for No Holiday)
    
    StoreType -> label encode
    Try label encoding first
    Potentially try 5 categories, meaning 4 one-hot columns
    Started with mode imputing
    
    Assortment -> label encode
    Try label encoding first
    Potentially try 4 categories, meaning 3 one-hot columns
    Started with mode imputing
    
    
**SAM**

    CompetitionOpenSinceMonth -> new int or float
    CompetitionOpenSinceYear -> new int or float
    Merge and create new feature, giving temporal distance from 2015-07-31 or 2015-08-31
    potentially use datetime for these calculations?
    Try: median imputation
    Try also: KNNImputer (sklearn)
    
    Promo2 -> one-hot encode
    3 categories including NaN, 2 columns
    make sure all missing in other Promo2 features are described by these three categories!
    
    Promo2SinceWeek -> new int or float
    Promo2SinceYear -> new int or float
    Merge and create new feature, giving temporal distance from 2015-07-31 or 2015-08-31
    potentially use datetime for these calculations?
    imputation with any value, this is caught by the boolean from Promo2 feature
    
    PromoInterval -> one-hot encode
    3 categories, NaNs can be added to the mode category, these will again be described by the Promo2 feature
    

### Imputing

**ATTENTION: After dropping all days with Sales=0, we only have days left where the store was opened, or where it was not known! Therefore changing NaNs for that feature to 0. We may completely delete the column later.**

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

zero_columns = ["Open", "Promo", "SchoolHoliday"]
zero_imputer = SimpleImputer(missing_values=np.nan, strategy="constant", fill_value=0)

mode_columns = ["StateHoliday", "StoreType", "Assortment"]
mode_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")

all_columns_ordered =  zero_columns + mode_columns

ct = ColumnTransformer(
    transformers=[
        ("zero_imputer", zero_imputer, zero_columns),
        ("mode_imputer", mode_imputer, mode_columns),
    ],
    verbose_feature_names_out=False,
)

transformed = ct.fit_transform(X_train)

Write imputed data to dataframe.

In [21]:
X_train_imputed = X_train.copy()
X_train_imputed.loc[:, all_columns_ordered] = transformed

Change 0.0 floats to "0" in StateHoliday.

In [22]:
def zerofun(row):
    if row["StateHoliday"] == "0.0":
        out = "0"
    else:
        out = row["StateHoliday"]
    return out

X_train_imputed = X_train_imputed.astype({"StateHoliday": str})

X_train_imputed.loc[:, "StateHoliday"] = X_train_imputed.apply(func=zerofun, axis=1)

In [23]:
X_train_imputed.head()

,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
143140,2013-06-12,353,3.0,1.0,0.0,0,0.0,b,b,900.0,NaN,NaN,1.0,14.0,2013.0,"Feb,May,Aug,Nov"
494300,2014-07-08,267,2.0,1.0,0.0,0,0.0,c,a,2460.0,1.0,2012.0,0.0,NaN,NaN,NaN
417249,2014-04-08,734,2.0,1.0,0.0,0,0.0,a,a,220.0,NaN,NaN,1.0,36.0,2013.0,"Mar,Jun,Sept,Dec"
370025,2014-02-17,0,1.0,1.0,1.0,0,0.0,a,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249373,2013-10-05,510,NaN,1.0,0.0,0,0.0,a,c,8260.0,NaN,NaN,0.0,NaN,NaN,NaN


### Encoding

Before encoding (or before imputing?) we should check our data types and change them where necessary.

In [24]:
X_train_imputed.dtypes

Date                          object
Store                          int64
DayOfWeek                    float64
Open                         float64
Promo                        float64
StateHoliday                  object
SchoolHoliday                float64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                       float64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [25]:
X_train_imputed = X_train_imputed.astype({
    "StateHoliday": "category",
    "StoreType": "category",
    "Assortment": "category"
})

Custom encoder. Still does not work with the sklearn.pipeline.Pipeline. Maybe we should turn the whole pipeline into a custom class?

In [26]:
class StoreTargetMeanEncoder():
    """Target mean encoder for Store feature in the Rossmann dataset.
    """
    
    def __init__(self):
        self.store_target_mean = None
    
    def fit(self, X, y):
        self._get_store_target_mean(feature=X, target=y)

    def transform(self, X):
        X = self._store_target_mean_encoder(X)
        return X
    
    def _get_store_target_mean(self, feature, target):
        all_data = pd.concat([X_train, y_train], axis=1)
        self.store_target_mean = all_data.groupby(by="Store").mean().loc[:, "Sales"].to_dict()
    
    def _store_target_mean_encoder(self, X):
        updated_store = X.loc[:, "Store"].map(self.store_target_mean)
        X_out = X.copy()
        X_out.update(updated_store)
        return X_out

Fitting and encoding using custom encoder.

In [27]:
# stme = StoreTargetMeanEncoder()
# stme.fit(X_train_imputed, y_train)
# stme.transform(X_train_imputed)

Encoding using the sklearn pipeline.

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

pipe = Pipeline([
    ('store_target_mean_encoder', StoreTargetMeanEncoder()),
    ('ordinal_encoder', OrdinalEncoder()),
])

my_features = ["Open", "Promo", "StateHoliday", "SchoolHoliday", "StoreType", "Assortment"]
# pipe.fit(X_train_imputed.loc[:, my_features], y_train)

Testing of encoder using specific features. We can use this encoder for a custom class in a custom pipeline, outside of a sklearn Pipeline instance.

In [29]:
oc = OrdinalEncoder()
oc.fit(X_train_imputed.loc[:, my_features], y_train)

OrdinalEncoder()

In [30]:
oc.transform(X_train_imputed.loc[:, my_features]).max(axis=0)

array([1., 1., 3., 1., 3., 2.])

In [31]:
X_train_imputed.loc[:, my_features].nunique()

Open             2
Promo            2
StateHoliday     4
SchoolHoliday    2
StoreType        4
Assortment       3
dtype: int64